In [67]:
import numpy as np
#

#
class Perceptron(object):
     
    def __init__(self, n_iterations=50, random_state=1, learning_rate=0.05):
        self.n_iterations = n_iterations
        self.random_state = random_state
        self.learning_rate = learning_rate
 
  
    def fit(self, X, y):
        rgen = np.random.RandomState(self.random_state)
        self.coef_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        for _ in range(self.n_iterations):
            for xi, expected_value in zip(X, y):
                predicted_value = self.predict(xi)
                self.coef_[1:] = self.coef_[1:] + self.learning_rate * (expected_value - predicted_value) * xi
                self.coef_[0] = self.coef_[0] + self.learning_rate * (expected_value - predicted_value) * 1
     
    
    def net_input(self, X):
            weighted_sum = np.dot(X, self.coef_[1:]) + self.coef_[0]
            return weighted_sum
     
  
    def activation_function(self, X):
            weighted_sum = self.net_input(X)
            return np.where(weighted_sum >= 0.0, 1, -1)
     
   
    def predict(self, X):
        return self.activation_function(X) 
     
  
    def score(self, X, y):
        misclassified_data_count = 0
        for xi, target in zip(X, y):
            output = self.predict(xi)
            if(target != output):
                misclassified_data_count += 1
        total_data_count = len(X)
        self.score_ = (total_data_count - misclassified_data_count)/total_data_count
        return self.score_
    

In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
import timeit



df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wpbc.data ',
                 header=None)

df[df[34] == '?'] = df[df[34] == '?'].replace({'?': '0'})
df[34] = df[34].astype(int)
df[1] = df[1].map({'N': -1, 'R': 1})

for col in df.columns:
    if col != 1:
        df[col] = (df[col] - df[col].mean()) / df[col].std()

print(df)

df_true = df[df[1] == 1]
df_false = df[df[1] == -1]

x_true = df_true.drop([1, 0, 2], axis=1)
y_true = df_true[1]

x_false = df_false.drop([1, 0, 2], axis=1)
y_false = df_false[1]
scores = []
cms = []
start = timeit.default_timer()
for i in [4, 8, 24]:  #
    x_train_true, x_test_true, y_train_true, y_test_true = train_test_split(x_true, y_true, test_size=0.3, random_state=i)
    x_train_false, x_test_false, y_train_false, y_test_false = train_test_split(x_false, y_false, test_size=0.3, random_state=i)
    x_train = x_train_true.append(x_train_false).reset_index().drop(["index"], axis=1)
    y_train = y_train_true.append(y_train_false).reset_index().drop(["index"], axis=1)
    x_test = x_test_true.append(x_test_false).reset_index().drop(["index"], axis=1)
    y_test = y_test_true.append(y_test_false).reset_index().drop(["index"], axis=1)
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
   
    
   

    model =Perceptron()
    for iteration in range(50):
        model.fit(x_train, y_train)
  
    score = model.score(x_test, y_test)
    scores.append(score)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    cms.append(cm) 
    
stop = timeit.default_timer()
print('perceptron Train time ', stop - start, ' sec')
print(scores)
print(cms[0])
print(cms[1])
print(cms[2])
print("mean: ", np.mean(scores))
print("std: ", np.std(scores))

           0   1         2         3         4         5         6         7   \
0   -0.647608  -1 -0.456501  0.192201  1.238630  0.123621  0.121991 -0.622197   
1   -0.686060  -1  0.414001  0.182712 -2.767615  0.371477  0.087915  1.255234   
2   -0.397349  -1  2.009922  1.251766 -1.125101  1.058926  1.144285 -1.143661   
3   -0.397015  -1  2.213039 -1.895300 -0.441108 -1.743248 -1.658220  3.179781   
4   -0.396980   1 -0.572568  0.910175 -1.846318  0.946689  0.928467 -0.190172   
..        ...  ..       ...       ...       ...       ...       ...       ...   
193 -0.362693  -1 -1.065852  1.615497 -0.082826  1.498519  1.780379 -2.137078   
194 -0.362405  -1 -1.123886 -0.623822  2.071519 -0.648006 -0.652113 -0.694068   
195 -0.656250  -1 -1.007819 -0.076644  1.608544 -0.227118 -0.155448 -1.051027   
196 -0.361113   1 -1.268969  1.267580  0.131213  1.409665  1.334545  0.344868   
197 -0.361014  -1 -1.181919 -0.225299  1.361935 -0.213089 -0.240355 -1.095747   

           8         9   ..